# Saliva Example

<div class="alert alert-block alert-info">
This example illustrates how to import saliva data (cortisol, amylase, etc.), how to compute often used parameters and how to export it to perform futher analysis.
</div>

## Setup and Helper Functions

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(
    context="notebook", 
    style="ticks", 
    font="sans-serif",
    palette=palette
)

plt.rcParams['figure.figsize'] = (8,4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

## Set Saliva Time Points

In [ ]:
sample_times = [-30, -1, 30, 40, 50, 60, 70]

## Load Condition List

Load example data

In [ ]:
condition_list = bp.example_data.get_condition_list_example()
condition_list.head()

Alternatively: Load your own data

In [ ]:
# condition_list = bp.io.load_subject_condition_list("<path-to-condition-list-file>")
# condition_list.head()

## Load Data

### Option 0: Load BioPsyKit example data 

The example data will be imported and converted into a long-format dataframe:

In [ ]:
df_cort = bp.example_data.get_saliva_example()
df_cort.head()

#### Compute Mean and Standard Error

Mean and standard error is computed for the two conditions and for each sample separately.

In [ ]:
bp.saliva.mean_se(df_cort, saliva_type="cortisol")

#### Plot Saliva Data

In [ ]:
bp.plotting.lineplot(data=df_cort, x="sample", y="cortisol", hue="condition", style="condition");

#### Example: Subject Exclusion

For this example, we assume we want to exclude the subjects 'Vp01' and 'Vp02' from both the condition list and the dataframe with cortisol samples

In [ ]:
dict_result = bp.utils.data_processing.exclude_subjects(
    ['Vp01', 'Vp02'], condition_list=condition_list, cortisol=df_cort
)

dict_result
# uncomment to reassign the data with excluded subjects to the original variable names
# df_cort = dict_result['cortisol']
# condition_list = dict_result['condition_list']

### Option 1: Use BioPsyKit to load saliva data in 'plate' format

The data is converted into long-format (`SalivaRawDataFrame`) and returned.

Load saliva data into pandas dataframe (example data using `biopsykit.example_data.get_saliva_example_plate_format()` or own data in 'plate'-format using `biopsykit.io.saliva.load_saliva_plate()`):

In [ ]:
df_cort = bp.example_data.get_saliva_example_plate_format()

# alternatively: load own data that is present in "plate" format
# df_cort = bp.io.saliva.load_saliva_plate(file_path="<path-to-saliva-data-in-plate-format>", saliva_type="cortisol")

In [ ]:
df_cort.head()

We can additionally directly pass a 'condition list' to the data loader function (for example data loader as well as for your own data). This allows us to directly assign subject conditions to the saliva samples:

In [ ]:
df_cort = bp.example_data.get_saliva_example_plate_format(condition_list=condition_list)
# alternatively: load own data in "plate" format and directly assign subject conditions
# df_cort = bp.io.saliva.load_saliva_plate(file_path="<path-to-saliva-data-in-plate-format>", saliva_type="cortisol", condition_list=condition_list)

df_cort.head()

Speficy your custom regular expression string to extract Subject ID and Saliva ID (see the documentation of `bp.saliva.io.load_saliva_plate()` for further information).

For example, the regex string in `regex_str` will extract the subject IDs **without** the `Vp` prefix and sample IDs **without** the `S` prefix:

In [ ]:
regex_str = "Vp(\d+) S(\d)"
df_cort = bp.example_data.get_saliva_example_plate_format(regex_str=regex_str)
# works analogously for bp.io.saliva.load_saliva_plate()

df_cort.head()

### Option 2: Use BioPsyKit to load saliva data that's already in the "correct" wide format

During import, the data is converted into long-format (`SalivaRawDataFrame`) and returned.

In [ ]:
df_cort = bp.example_data.get_saliva_example()
# Alternatively, use your own data:
# df_cort = bp.io.saliva.load_saliva_wide_format(file_path="<path-to-cortisol-data-in-wide-format.csv>", saliva_type="cortisol")

In [ ]:
df_cort.head()

## Save Data

Save Dataframe as csv (in standardized format)

In [ ]:
#bp.io.saliva.save_saliva("<export-path.csv>", df_cort)

## Saliva Data Processing

Saliva Data in `SalivaRawDataFrame`

In [ ]:
df_cort.head()

### Mean and Standard Error over all Subjects

Computing mean and standard error over all subjects returns a `SalivaMeanSeDataFrame`. This dataframe can directly be used for plotting functions from `BioPsyKit`, such as `biopsykit.protocols.plotting.saliva_plot()` (more on that later).

In [ ]:
df_cort_mean_se = bp.saliva.mean_se(df_cort)
df_cort_mean_se.head()

### Saliva Features

#### Standard Features

Compute a set of "Standard Features", including:
* `argmax`: location of maximum
* `mean`: mean value
* `std`: standard deviation
* `skew`: skewness
* `kurt`: kurtosis

In [ ]:
bp.saliva.standard_features(df_cort).head()

#### AUC

Area under the Curve (AUC), in different variations (according to Pruessner et al. 2003):
* `auc_g`: Total Area under the Curve
* `auc_i`: Area under the Curve with respect to increae
* `auc_i_post`: (if `compute_auc_post=True`) Area under the Curve with respect to increase *after* the stressor: This is only relevant for an acute stress scenario where we collected saliva samples before and after the stressor. Per `BioPsyKit` convention, saliva samples collected *after* the stressor have saliva times $t \geq 0$.

**Note**: For these examples we neglect the first saliva sample (S0) by setting `remove_s0` to `True` because this sample was only assessed to control for high cortisol baseline and is not relevant for feature computation. The feature computation is performed on the remaining saliva samples (S1-S6).

In [ ]:
# sample times must directly be supplied if they are not already part of the saliva dataframe
bp.saliva.auc(df_cort, remove_s0=True, sample_times=sample_times).head()

#### Maximum Increase

Absolute maximum increase (or the relative increase in percent if `percent=True`) between the *first* sample in the data and *all others*:

In [ ]:
bp.saliva.max_increase(df_cort, remove_s0=True).head()

#### Slope

Slope between two saliva samples (specified by `sample_idx`):

In [ ]:
bp.saliva.slope(df_cort, sample_idx=(1, 4), sample_times=sample_times).head()

## Plotting

### Using Seaborn (some very simple Examples)

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(
    data=df_cort.reset_index(), 
    x="sample", 
    y="cortisol", 
    hue="condition", 
    hue_order=["Control", "Intervention"], 
    ci=None, # ci = None => no error bars
    ax=ax
)
ax.set_ylabel("Cortisol [nmol/l]")
ax.set_xlabel("Sample Times")
fig.tight_layout()

In [ ]:
sns.relplot(
    data=df_cort.reset_index(), 
    x="sample", 
    y="cortisol", 
    kind="line", 
    hue="condition", 
    hue_order=["Control", "Intervention"]
)

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(
    data=df_cort.reset_index(), 
    x="sample", 
    y="cortisol", 
    hue="condition", 
    hue_order=["Control", "Intervention"], 
    ax=ax
)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(
    data=bp.saliva.max_increase(df_cort).reset_index(), 
    x="condition", 
    y="cortisol_max_inc", 
    order=["Control", "Intervention"]
)

In [ ]:
fig, ax = plt.subplots()

data_features = bp.saliva.utils.saliva_feature_wide_to_long(bp.saliva.standard_features(df_cort), "cortisol")

sns.boxplot(
    data=data_features.reset_index(), 
    x="saliva_feature", 
    y="cortisol", 
    hue="condition", 
    hue_order=["Control", "Intervention"], 
    ax=ax
)
fig.tight_layout()

### Using functions from `BioPsyKit`

In [ ]:
df_cort_mean_se.T

**Note**: This example shows how to use the function `biopsykit.protocols.plotting.saliva_plot()`. If you recorded saliva data within a psychological protoco, however, it's recommended to use the Protocol API and create a `Protocol` object, where you can add all data and use plotting functions in a more convenient way.

#### Saliva Plot

In [ ]:
# plot without first saliva sample
bp.protocols.plotting.saliva_plot(
    df_cort_mean_se.drop("0", level="sample"), 
    saliva_type="cortisol", 
    sample_times=sample_times[1:], 
    sample_times_absolute=True,
    test_times=[0, 30],
    test_title="TEST"
)

#### Saliva Features

All features in one plot, `x` variable separates the features, `hue` variable separates the conditions.

In [ ]:
fig, ax = plt.subplots()
bp.protocols.plotting.saliva_feature_boxplot(
    data=data_features, 
    x="saliva_feature", 
    saliva_type="cortisol", 
    hue="condition", 
    hue_order=["Control", "Intervention"],
    ax=ax
)
fig.tight_layout()

One subplot per feature, `x` variable separates the conditions. `features` is provided a list of the features to plot.

In [ ]:
bp.protocols.plotting.saliva_multi_feature_boxplot(
    data=data_features, 
    saliva_type="cortisol", 
    features=["argmax", "kurt", "std", "mean"],
    hue="condition", 
    hue_order=["Control", "Intervention"],
)
fig.tight_layout()

Grouping of features per subplot, `features` is provided a dictionary that specifies the mapping.

In [ ]:
bp.protocols.plotting.saliva_multi_feature_boxplot(
    data=data_features, 
    saliva_type="cortisol", 
    features={"mean": ["mean", "argmax"], "std": ["std", "skew", "kurt"]},
    hue="condition", 
    hue_order=["Control", "Intervention"],
)
fig.tight_layout()